In [1]:
!pip install gurobipy>=9.5.1

In [2]:
import gurobipy as gp
from gurobipy import GRB as GRB
import sys
sys.path.insert(0, "../utils")

import utils_functions as uf
import unavailability

In [3]:
T = range(1,171)
ab = uf.calculate_unavailability(unavailability.all_group_est_unavailability)

b = {}
for n, p in enumerate(ab):
    b[n+1] = {}
    for t in T:
        if t in p:
            b[n+1][t] = 1
        else:
            b[n+1][t] = 0

In [4]:
# Setup Gurobi key
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '453f2da9-b8f2-448f-9f06-48081a9c3dc9')
e.setParam('WLSSECRET', 'd5675c81-7058-45f9-b6f2-30ca3855ed22')
e.setParam('LICENSEID', 868440)
e.start()

# Create the model within the Gurobi environment. This environment will be passed in as a parameter into all subsequent models
model = gp.Model(env=e)
model.setParam('Seed', 435)

Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868440
Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Set parameter Seed to value 435


In [5]:
C = range(1,11)
M = range(11,41)
G = range(1,3)
I = range(1,41)
BIGM = 100

In [6]:
p = model.addVars(I,G, vtype=GRB.BINARY, name="p")
x = model.addVars(G,T, vtype=GRB.BINARY, name="x")

In [7]:
model.setObjective(sum(x[g,t] for g in G for t in T), sense=GRB.MINIMIZE)

In [8]:
model.addConstrs(sum(b[i][t]*p[i,g] for i in I)  <= BIGM*x[g,t] for g in G for t in T)
model.addConstrs(sum(p[i,g] for g in G) == 1 for i in I)
model.addConstrs(sum(p[c,g] for c in C) >= 5 for g in G)
model.addConstrs(sum(p[m,g] for m in M) >= 15 for g in G)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [9]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Optimize a model with 384 rows, 420 columns and 2866 nonzeros
Model fingerprint: 0xffa16d69
Variable types: 0 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 261.0000000
Presolve removed 202 rows and 201 columns
Presolve time: 0.01s
Presolved: 182 rows, 219 columns, 1235 nonzeros
Variable types: 0 continuous, 219 integer (218 binary)

Root relaxation: objective 1.760000e+02, 456 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bound

In [13]:

for g in G:
    ts = 0
    for i in I:
        if p[i,g].X == 1:
            ts += 1
            print(f'\tTeam {g} has person {i}')
    print(f'Team {g} is busy for {sum([x[g,t].X for t in T])} time slots')
    print(f'Team {g} has size {ts}')

	Team 1 has person 4
	Team 1 has person 6
	Team 1 has person 8
	Team 1 has person 9
	Team 1 has person 10
	Team 1 has person 13
	Team 1 has person 14
	Team 1 has person 17
	Team 1 has person 18
	Team 1 has person 19
	Team 1 has person 20
	Team 1 has person 21
	Team 1 has person 25
	Team 1 has person 27
	Team 1 has person 28
	Team 1 has person 31
	Team 1 has person 37
	Team 1 has person 38
	Team 1 has person 39
	Team 1 has person 40
Team 1 is busy for 97.0 time slots
Team 1 has size 20
	Team 2 has person 1
	Team 2 has person 2
	Team 2 has person 3
	Team 2 has person 5
	Team 2 has person 7
	Team 2 has person 11
	Team 2 has person 12
	Team 2 has person 15
	Team 2 has person 16
	Team 2 has person 22
	Team 2 has person 23
	Team 2 has person 24
	Team 2 has person 26
	Team 2 has person 29
	Team 2 has person 30
	Team 2 has person 32
	Team 2 has person 33
	Team 2 has person 34
	Team 2 has person 35
	Team 2 has person 36
Team 2 is busy for 113.0 time slots
Team 2 has size 20
